In [1]:
%%spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1636125135430_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f

{    "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "driverMemory": "14000M",
          "spark.driver.memory": "14000M",
          "spark.executor.memory": "5000M", 
          "maximizeResourceAllocation": "true",
          "spark.sql.crossJoin.enabled": "true",
          "spark.network.timeout": "8500",  
          "spark.sql.catalogImplementation":"hive",
          "hive.metastore.client.factory.class": "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory",
          "spark.jars":"s3://geolab-datamart/QA/geolab-files/tasaciones/emr/geospark_jars/geo_wrapper_2.11-0.3.0.jar,s3://geolab-datamart/QA/geolab-files/tasaciones/emr/geospark_jars/geospark-1.3.0.jar,s3://geolab-datamart/QA/geolab-files/tasaciones/emr/geospark_jars/geospark-sql_2.3-1.3.0.jar,s3://geolab-datamart/QA/geolab-files/tasaciones/emr/geospark_jars/sedona-core-3.0_2.12-1.0.1-incubating.jar,s3://geolab-datamart/QA/geolab-files/tasaciones/emr/geospark_jars/sedona-sql-3.0_2.12-1.0.1-incubating.jar,s3://geolab-datamart/QA/geolab-files/tasaciones/emr/geospark_jars/sedona-viz-3.0_2.12-1.0.1-incubating.jar"
     }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1636125135430_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1636125135430_0006,pyspark,idle,Link,Link,✔


In [3]:
sc.install_pypi_package("Cython")
sc.install_pypi_package("pandas==1.2.0")
sc.install_pypi_package("pyarrow==2")
sc.install_pypi_package("awswrangler")
sc.install_pypi_package("shapely")
sc.install_pypi_package("geospark")
# sc.install_pypi_package("apache-sedona")
# sc.uninstall_package("")
print(sc.list_packages())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/3d/48/bbca549da0b0f636c0f161e84d30172c40aafe99552680f297da7fedf102/Cython-0.29.24-cp37-cp37m-manylinux1_x86_64.whl

  Using cached https://files.pythonhosted.org/packages/ff/bd/fb376f9fbad92b9a6efdbb30ff32c80f3cba1368689309cbb5566364af5c/pandas-1.2.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/c8/58/d07e7ee8b0cffe509f9e5a3742e09636a4a58b2113d193166615b934846f/pyarrow-2.0.0-cp37-cp37m-manylinux1_x86_64.whl

  Using cached https://files.pythonhosted.org/packages/64/ba/9ab7116c339154ca148d65bdfd3785f41f113631c6e792d8de7b45883423/awswrangler-2.12.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6b/a2/e393d9c933e6312854386008e6d93435df978e20b7d672bb78c4b72bf649/pg8000-1.21.3-py3-none-any.whl
  Using cached

In [4]:
import pandas as pd
import numpy as np
import awswrangler as wr 
print(np.__version__ , pd.__version__ , wr.__version__)

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# import pyspark.sql.function as f

from geospark.register import GeoSparkRegistrator
GeoSparkRegistrator.registerAll(spark)
 
# from geospark.utils.adapter import Adapter
# from geospark.core.enums import GridType, IndexType 
# from geospark.core.spatialOperator  import JoinQuery, KNNQuery, RangeQuery

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.16.5 1.2.0 2.12.1
True

In [5]:
def create_and_cache_sdf(sdf, view_name): 
    sdf.cache().createOrReplaceTempView( view_name )
#     print(f"Registros en {view_name} :",sdf.count())
    print(sdf.show(2,False))
    print(sdf.printSchema())
    print(sqlContext.tableNames("default"))
     
## Guardar a S3 
def save_on_s3( sdf,table_name, format = 'parquet'):
    
    s3_target_path  = f"s3://geolab-datamart/raw/inputs_estudios/outputs/{table_name}/"
    print(f"Saving as {format} on : {s3_target_path}" )
    
    if format == 'csv' :
        # se indica que quermos 1 archivo por particion
        sdf.repartition(1).write.option("header", "true").option("delimiter", ";").format("csv").mode("overwrite").save(s3_target_path)
    if format =='parquet' : 
        # se indica que quermos 1 archivo por particion
        sdf.repartition(1).write.format("parquet").mode("overwrite").save(s3_target_path)
        
    print(f"Saved !") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
env = "QA"  #"PROD"
region = "us-east-1"
project_name = "inputs_estudios" 
# table_name = "buffers_layers"
s3_datamart_base_path = f"s3://geolab-datamart/raw/inputs_estudios/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
dominios = [ 'co','pe' ]
buffers = [ 1500  ,  3000]
canastas = []

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Creación de Buffers

In [23]:
for dominio in dominios :   
    for buffer in buffers :   
        
        ## SE CREA LA TABLA DE BLOCKS
        table = "blocks"
        s3_blocks_path =f"{s3_datamart_base_path}/country_{dominio}_{table}.csv"  
        sdf_blocks = spark.read.csv(s3_blocks_path, header=True, sep=';'  ) 
        view_name = f"{table}_{dominio}"
        create_and_cache_sdf(sdf_blocks , view_name)
        #blocks_co
        
        ## SE CREA LA TABLA DE BUFFERS 
        sdf_blocks_shapes = spark.sql(f"""
        WITH BASE AS (
            SELECT   
                geo_id,
                administrative_area_level_1,   
                ST_Transform(ST_Point(CAST(longitud AS DECIMAL(24,14))  , CAST(latitud AS DECIMAL(24,14)) ) , 'epsg:4326', 'epsg:32719') planar_coord 
                  
            FROM blocks_{dominio} B
        )
        SELECT  
            geo_id,   
            --ST_AsText(ST_Transform(ST_Buffer(planar_coord, 1500.0),  'epsg:32719','epsg:4326'))  as buffer_1500, 
            --ST_AsText(ST_Transform(ST_Buffer(planar_coord, 3000.0),  'epsg:32719','epsg:4326'))  as buffer_3000
            ST_Transform(ST_Buffer(planar_coord, 1500.0),  'epsg:32719','epsg:4326')  as buffer_1500, 
            ST_Transform(ST_Buffer(planar_coord, 3000.0),  'epsg:32719','epsg:4326')  as buffer_3000
        FROM BASE  
        """)

        view_name2=f"blocks_shapes_{dominio}"
        create_and_cache_sdf(sdf_blocks_shapes ,  view_name2)
#         save_on_s3( sdf_block_shapes,view_name2, format = 'csv')

#         break
#     break

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+-------------+------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+------------+----------------+----------+---------+-----------+----------------+---------+------------+-------------+-------------+-------------+-------------+-------------------+-------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |geo_id|area         |perimeter   |administrative_area_level_1|administrative_area_level_2|administrative_area_level_3|administrative_area_level_4|administrative_area_level_5|income_level|total_households|density   |total_men|total_women|total_population|recoba_id|latitud     |longitud     |pxq          |pxq_density  |pxq_average  |pxq_density_average|var_cen|date               |shape                     

In [12]:
print(sqlContext.tableNames("default"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['blocks_co', 'blocks_pe', 'blocks_shapes_co', 'blocks_shapes_pe']

## ETL IG_Demográfico

In [13]:
sdf_block_shapes.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geo_id: string (nullable = true)
 |-- buffer_1500: string (nullable = false)
 |-- buffer_3000: string (nullable = false)

In [26]:
for dominio in dominios :   
    
    # CARGA TABLA DE VIEW_BLOCKS
    table = "view_blocks" 
    sdf_view_blocks = spark.read.csv(f"{s3_datamart_base_path}/country_{dominio}_{table}.csv" , header=True, sep=';'  ) 
    view_name = f"{table}_{dominio}"
    create_and_cache_sdf(sdf_view_blocks , view_name)

    # CARGA DE BUFFERS CORRESPONDIENTES 
#     table = f"blocks_shapes_{dominio}" 
#     sdf_blocks_shapes = spark.read.csv(f"{s3_datamart_base_path}/outputs/block_shapes_{dominio}/*.csv" , header=True, sep=';'  ) 

#     create_and_cache_sdf(sdf_blocks_shapes , table)
    
    
    for buffer in buffers :   
        #print(dominio, buffer) 
        ## SE CREA LA TABLA DE BUFFERS
        sdf_demografico = spark.sql(f"""
        WITH INTERSECCION AS (
            SELECT
                B.geo_id,
                VB.hog_gse1 AS gse1,
                VB.hog_gse2 AS gse2,
                VB.hog_gse3 AS gse3,
                VB.hog_gse4 AS gse4,
                VB.hog_gse5 AS gse5,
                VB.hog_gse6 AS gse6,
                VB.hog_gse7 AS gse7,
                VB.pob_gse1 AS pob_gse1,
                VB.pob_gse2 AS pob_gse2,
                VB.pob_gse3 AS pob_gse3,
                VB.pob_gse4 AS pob_gse4,
                VB.pob_gse5 AS pob_gse5,
                VB.pob_gse6 AS pob_gse6,
                VB.pob_gse7 AS pob_gse7,
                VB.pob_edad1 AS pob_edad1,
                VB.pob_edad2 AS pob_edad2,
                VB.pob_edad3 AS pob_edad3,
                VB.pob_edad4 AS pob_edad4,
                VB.pob_edad5 AS pob_edad5,
                VB.pob_edad6 AS pob_edad6,
                VB.pob_edad7 AS pob_edad7,
                VB.pob_edad8 AS pob_edad8,
                VB.pob_edad9 AS pob_edad9,
                VB.pob_edad10 AS pob_edad10,
                VB.pob_edad11 AS pob_edad11,
                VB.pob_edad12 AS pob_edad12,
                VB.pob_edad13 AS pob_edad13,
                VB.pob_edad14 AS pob_edad14,
                VB.pob_edad15 AS pob_edad15,
                VB.pob_edad16 AS pob_edad16,
                VB.pob_edad17 AS pob_edad17,
                VB.pob_edad18 AS pob_edad18,
                VB.pob_edad19 AS pob_edad19,
                VB.pob_edad20 AS pob_edad20,
                VB.pob_edad21 AS pob_edad21,
                VB.total_men AS total_men,
                VB.total_women AS total_women,
                VB.total_households AS total_households,
                VB.total_population AS total_population,
                VB.pxq AS pxq 

            FROM  view_blocks_{dominio} VB
            JOIN    blocks_shapes_{dominio} B
            ON ST_intersects( B.buffer_{buffer}  , st_geomFromWKT(VB.shape))
         
        )
       SELECT 
            SUM(gse1),
            SUM(gse2),
            SUM(gse3),
            SUM(gse4),
            SUM(gse5),
            SUM(gse6),
            SUM(gse7),
            SUM(pob_gse1),
            SUM(pob_gse2),
            SUM(pob_gse3),
            SUM(pob_gse4),
            SUM(pob_gse5),
            SUM(pob_gse6),
            SUM(pob_gse7),
            SUM(pob_edad1),
            SUM(pob_edad2),
            SUM(pob_edad3),
            SUM(pob_edad4),
            SUM(pob_edad5),
            SUM(pob_edad6),
            SUM(pob_edad7),
            SUM(pob_edad8),
            SUM(pob_edad9),
            SUM(pob_edad10),
            SUM(pob_edad11),
            SUM(pob_edad12),
            SUM(pob_edad13),
            SUM(pob_edad14),
            SUM(pob_edad15),
            SUM(pob_edad16),
            SUM(pob_edad17),
            SUM(pob_edad18),
            SUM(pob_edad19),
            SUM(pob_edad20),
            SUM(pob_edad21),
            SUM(total_men),
            SUM(total_women),
            SUM(total_households),
            SUM(total_population) 
            FROM INTERSECCION
        GROUP BY
            geo_id
            
        """)
        
#         view_name2=f"intersect_demografico_{dominio}_b{buffer}"
#         create_and_cache_sdf(sdf_demografico ,  view_name2) 
# #         save_on_s3( sdf_block_shapes,view_name2, format = 'csv')
         
        view_name2=f"customer_country_{dominio}_empresas_b{buffer}"
        create_and_cache_sdf(sdf_demografico ,  view_name2) 
#         save_on_s3( sdf_block_shapes,view_name2, format = 'csv')

        break
    break

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://11.0.5.166:18888/sessions/5/statements/24 with error payload: {"msg":"requirement failed: Session isn't active."}


In [14]:
dominio , buffer =  'co', 1500

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
table = "view_blocks" 
sdf_view_blocks = spark.read.csv(f"{s3_datamart_base_path}/country_{dominio}_{table}.csv" , header=True, sep=';'  ) 
view_name = f"{table}_{dominio}"
create_and_cache_sdf(sdf_view_blocks , view_name)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------+------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+------------+----------------+----------+---------+-----------+----------------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-------------+-------------+-------------+-------------+-----------+--------------

In [16]:


sdf_demografico = spark.sql(f""" 

SELECT 
    B.geo_id,
    CAST(VB.hog_gse1 AS DECIMAL) AS gse1,
    CAST(VB.hog_gse2 AS DECIMAL)  AS gse2,
    CAST(VB.hog_gse3 AS DECIMAL)  AS gse3,
    CAST(VB.hog_gse4 AS DECIMAL)  AS gse4,
    CAST(VB.hog_gse5 AS DECIMAL)  AS gse5,
    CAST(VB.hog_gse6 AS DECIMAL)  AS gse6,
    VB.hog_gse7 AS gse7,
    VB.pob_gse1 AS pob_gse1,
    VB.pob_gse2 AS pob_gse2,
    VB.pob_gse3 AS pob_gse3,
    VB.pob_gse4 AS pob_gse4,
    VB.pob_gse5 AS pob_gse5,
    VB.pob_gse6 AS pob_gse6,
    VB.pob_gse7 AS pob_gse7,
    VB.pob_edad1 AS pob_edad1,
    VB.pob_edad2 AS pob_edad2,
    VB.pob_edad3 AS pob_edad3,
    VB.pob_edad4 AS pob_edad4,
    VB.pob_edad5 AS pob_edad5,
    VB.pob_edad6 AS pob_edad6,
    VB.pob_edad7 AS pob_edad7,
    VB.pob_edad8 AS pob_edad8,
    VB.pob_edad9 AS pob_edad9,
    VB.pob_edad10 AS pob_edad10,
    VB.pob_edad11 AS pob_edad11,
    VB.pob_edad12 AS pob_edad12,
    VB.pob_edad13 AS pob_edad13,
    VB.pob_edad14 AS pob_edad14,
    VB.pob_edad15 AS pob_edad15,
    VB.pob_edad16 AS pob_edad16,
    VB.pob_edad17 AS pob_edad17,
    VB.pob_edad18 AS pob_edad18,
    VB.pob_edad19 AS pob_edad19,
    VB.pob_edad20 AS pob_edad20,
    VB.pob_edad21 AS pob_edad21,
    VB.total_men AS total_men,
    VB.total_women AS total_women,
    VB.total_households AS total_households,
    VB.total_population AS total_population,
    VB.pxq AS pxq
FROM  view_blocks_{dominio} VB,
JOIN    blocks_shapes_{dominio} B
ON ST_intersects(st_geomFromWKT(B.buffer_{buffer}), st_geomFromWKT(VB.shape)) 


""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
sdf_demografico

In [17]:
sdf_demografico.show(100 , False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----+----+----+----+----+----+----+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+-----------+----------------+----------------+---+
|geo_id|gse1|gse2|gse3|gse4|gse5|gse6|gse7|pob_gse1|pob_gse2|pob_gse3|pob_gse4|pob_gse5|pob_gse6|pob_gse7|pob_edad1|pob_edad2|pob_edad3|pob_edad4|pob_edad5|pob_edad6|pob_edad7|pob_edad8|pob_edad9|pob_edad10|pob_edad11|pob_edad12|pob_edad13|pob_edad14|pob_edad15|pob_edad16|pob_edad17|pob_edad18|pob_edad19|pob_edad20|pob_edad21|total_men|total_women|total_households|total_population|pxq|
+------+----+----+----+----+----+----+----+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----

In [ ]:
  
        SELECT 
            SUM(gse1),
            SUM(gse2),
            SUM(gse3),
            SUM(gse4),
            SUM(gse5),
            SUM(gse6),
            SUM(gse7),
            SUM(pob_gse1),
            SUM(pob_gse2),
            SUM(pob_gse3),
            SUM(pob_gse4),
            SUM(pob_gse5),
            SUM(pob_gse6),
            SUM(pob_gse7),
            SUM(pob_edad1),
            SUM(pob_edad2),
            SUM(pob_edad3),
            SUM(pob_edad4),
            SUM(pob_edad5),
            SUM(pob_edad6),
            SUM(pob_edad7),
            SUM(pob_edad8),
            SUM(pob_edad9),
            SUM(pob_edad10),
            SUM(pob_edad11),
            SUM(pob_edad12),
            SUM(pob_edad13),
            SUM(pob_edad14),
            SUM(pob_edad15),
            SUM(pob_edad16),
            SUM(pob_edad17),
            SUM(pob_edad18),
            SUM(pob_edad19),
            SUM(pob_edad20),
            SUM(pob_edad21),
            SUM(total_men),
            SUM(total_women),
            SUM(total_households),
            SUM(total_population) 
            FROM BASE
        GROUP BY
            geo_id  

In [21]:
dominio = 'co'
buffer=1500
sdf_demografico = spark.sql(f""" 
    WITH BASE AS (
    SELECT 
        B.geo_id,
        VB.hog_gse1 AS gse1,
        VB.hog_gse2 AS gse2,
        VB.hog_gse3 AS gse3,
        VB.hog_gse4 AS gse4,
        VB.hog_gse5 AS gse5,
        VB.hog_gse6 AS gse6,
        VB.hog_gse7 AS gse7,
        VB.pob_gse1 AS pob_gse1,
        VB.pob_gse2 AS pob_gse2,
        VB.pob_gse3 AS pob_gse3,
        VB.pob_gse4 AS pob_gse4,
        VB.pob_gse5 AS pob_gse5,
        VB.pob_gse6 AS pob_gse6,
        VB.pob_gse7 AS pob_gse7,
        VB.pob_edad1 AS pob_edad1,
        VB.pob_edad2 AS pob_edad2,
        VB.pob_edad3 AS pob_edad3,
        VB.pob_edad4 AS pob_edad4,
        VB.pob_edad5 AS pob_edad5,
        VB.pob_edad6 AS pob_edad6,
        VB.pob_edad7 AS pob_edad7,
        VB.pob_edad8 AS pob_edad8,
        VB.pob_edad9 AS pob_edad9,
        VB.pob_edad10 AS pob_edad10,
        VB.pob_edad11 AS pob_edad11,
        VB.pob_edad12 AS pob_edad12,
        VB.pob_edad13 AS pob_edad13,
        VB.pob_edad14 AS pob_edad14,
        VB.pob_edad15 AS pob_edad15,
        VB.pob_edad16 AS pob_edad16,
        VB.pob_edad17 AS pob_edad17,
        VB.pob_edad18 AS pob_edad18,
        VB.pob_edad19 AS pob_edad19,
        VB.pob_edad20 AS pob_edad20,
        VB.pob_edad21 AS pob_edad21,
        VB.total_men AS total_men,
        VB.total_women AS total_women,
        VB.total_households AS total_households,
        VB.total_population AS total_population,
        VB.pxq AS pxq
    FROM  view_blocks_{dominio} VB,
         blocks_shapes_{dominio} B
    WHERE ST_intersects(B.buffer_{buffer}, VB.shape)
    )

    

    """)


# sdf_demografico

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
sdf_view_blocks.show(100 , False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------+-------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+------------+----------------+----------+---------+-----------+----------------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-------------+-------------+-------------+-------------+-----------+-------------

In [20]:
spark.sql(f""" SELECT  * FROM intersect_demografico_co_b1500 limit 100 """).show(100 , False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----+----+----+----+----+----+----+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+-----------+----------------+----------------+---+
|geo_id|gse1|gse2|gse3|gse4|gse5|gse6|gse7|pob_gse1|pob_gse2|pob_gse3|pob_gse4|pob_gse5|pob_gse6|pob_gse7|pob_edad1|pob_edad2|pob_edad3|pob_edad4|pob_edad5|pob_edad6|pob_edad7|pob_edad8|pob_edad9|pob_edad10|pob_edad11|pob_edad12|pob_edad13|pob_edad14|pob_edad15|pob_edad16|pob_edad17|pob_edad18|pob_edad19|pob_edad20|pob_edad21|total_men|total_women|total_households|total_population|pxq|
+------+----+----+----+----+----+----+----+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----

In [43]:

spark.sql(f""" SELECT  * FROM blocks_shapes_co limit 1000 """).printSchema()#.show(100 , False).printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geo_id: string (nullable = true)
 |-- buffer_1500: string (nullable = false)
 |-- buffer_3000: string (nullable = false)

In [22]:
 spark.sql(f"""  SELECT  shape FROM view_blocks_CO LIMIT 100""").show(100 , False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
 spark.sql(f"""  SELECT  * FROM customer_country_pe_empresas_b3000 LIMIT 100""").show(100 , False)